# Dataset Information  
Name: DOHMH Dog Bite Data  
Author: New York City Department of Health and Mental Hygiene (NYC DOHMH)  
Source: https://data.cityofnewyork.us/Health/DOHMH-Dog-Bite-Data/rsgh-akpg/about_data  
Accessed: 2024 November 2  
Method of Data Collection:  
* Reports received online, mail, fax or by phone to 311
* NYC DOHMH Animal Bite Unit

# Feature Information
<table style='margin-left: auto; margin-right: auto'>
    <tr>
        <th colspan='3'> DOHMH Dog Bite Data </th>
    <tr>
    <tr>
        <th> Column Name </th>
        <th> Description </th>
        <th> Data Type </th>
    </tr>
    <tr>
        <td> UniqueID </th>
        <td> Unique dog bite case identifier </th>
        <td> Text </th>
    </tr>
    <tr>
        <td> DateOfBite </th>
        <td> Date bitten </th>
        <td> Floating Timestamp </th>
    </tr>
    <tr>
        <td> Species </th>
        <td> Animal Type (Dog) </th>
        <td> Text </th>
    </tr>
    <tr>
        <td> Breed </th>
        <td> Breed type </th>
        <td> Text </th>
    </tr>
        <tr>
        <td> Age </th>
        <td> Dog's age at time of bite. Numbers with 'M' indicate months. </th>
        <td> Text </th>
    </tr>
    <tr>
        <td> Gender </th>
        <td> Sex of Dog. M=Male, F=Female, U=Unknown </th>
        <td> Text </th>
    </tr>
    <tr>
        <td> SpayNeuter </th>
        <td> Surgical removal of dog's reproductive organs. True (reported to DOHMH as Spayed or Neutered), False (Unknown or Not Spayed or Neutered) </th>
        <td> Boolean </th>
    </tr>
    <tr>
        <td> Borough </th>
        <td> Dog bite Borough. 'Other' indicates that the bite took place outside New York City </th>
        <td> Text </th>
    </tr>
    <tr>
        <td> ZipCode </th>
        <td> Dog bite Zipcode. Blank ZipCode indicates that information was not available </th>
        <td> Text </th>
    </tr>
</table>

# Import and Initializing Cleaning

In [146]:
# libraries
import pandas as pd
from utils import breed_mapping, useless_breed_words

In [147]:
# import data
raw = pd.read_csv('../data/raw/DOHMH_Dog_Bite_Data_20241102.csv')

# initiliaze cleaned data
cleaned = raw.copy()

# display
raw.head()

,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode
0,1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220
1,2,January 04 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,NaN
2,3,January 06 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224
3,4,January 08 2018,DOG,Mixed/Other,4,M,False,Brooklyn,11231
4,5,January 09 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224


In [148]:
# snake case column names
snake_case = {
    'UniqueID': 'unique_id',
    'DateOfBite': 'date_of_bite',
    'SpayNeuter': 'spay_neuter',
    'ZipCode': 'zip_code',
}

cleaned.rename(columns=snake_case, inplace=True)
cleaned.rename(columns=str.lower, inplace=True)

# display
cleaned.head()

,unique_id,date_of_bite,species,breed,age,gender,spay_neuter,borough,zip_code
0,1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220
1,2,January 04 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,NaN
2,3,January 06 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224
3,4,January 08 2018,DOG,Mixed/Other,4,M,False,Brooklyn,11231
4,5,January 09 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224


In [149]:
# drop columns
# UniqueID: not useful
# Species: only has one value (dog)
# Age: too many missing values
# Gender: too many missing values

cleaned = cleaned.drop(columns=['unique_id', 'species', 'age', 'gender'])

# display
cleaned.head()

,date_of_bite,breed,spay_neuter,borough,zip_code
0,January 01 2018,UNKNOWN,False,Brooklyn,11220
1,January 04 2018,UNKNOWN,False,Brooklyn,NaN
2,January 06 2018,Pit Bull,False,Brooklyn,11224
3,January 08 2018,Mixed/Other,False,Brooklyn,11231
4,January 09 2018,Pit Bull,False,Brooklyn,11224


In [150]:
# convert date_of_bite to datetime
cleaned['date_of_bite'] = pd.to_datetime(cleaned['date_of_bite'])

# convert spay_neuter to boolean
cleaned['spay_neuter'] = cleaned['spay_neuter'].astype('bool')

# lower case all string columns
string_columns = cleaned.select_dtypes(include='object').columns
cleaned[string_columns] = cleaned[string_columns].apply(lambda x: x.str.lower())

# convert all nan into None
cleaned = cleaned.where(pd.notnull(cleaned), None)

# display
cleaned.head()

,date_of_bite,breed,spay_neuter,borough,zip_code
0,2018-01-01,unknown,False,brooklyn,11220
1,2018-01-04,unknown,False,brooklyn,None
2,2018-01-06,pit bull,False,brooklyn,11224
3,2018-01-08,mixed/other,False,brooklyn,11231
4,2018-01-09,pit bull,False,brooklyn,11224


In [151]:
# check for missing values
cleaned.isna().sum()

date_of_bite       0
breed           2263
spay_neuter        0
borough            0
zip_code        7167
dtype: int64

# Wrangling Date of Bite
Extract date values

In [152]:
# extract date values
cleaned['year'] = pd.to_datetime(cleaned['date_of_bite']).dt.year
cleaned['month'] = pd.to_datetime(cleaned['date_of_bite']).dt.month
cleaned['day'] = pd.to_datetime(cleaned['date_of_bite']).dt.day
cleaned['day_of_week'] = pd.to_datetime(cleaned['date_of_bite']).dt.dayofweek

# drop date_of_bite
cleaned.drop('date_of_bite', axis=1, inplace=True)

# display
cleaned.head()

,breed,spay_neuter,borough,zip_code,year,month,day,day_of_week
0,unknown,False,brooklyn,11220,2018,1,1,0
1,unknown,False,brooklyn,None,2018,1,4,3
2,pit bull,False,brooklyn,11224,2018,1,6,5
3,mixed/other,False,brooklyn,11231,2018,1,8,0
4,pit bull,False,brooklyn,11224,2018,1,9,1


# Wrangling Zip Code
Fill missing with mode of zip code from each borough  
Must be in integer

In [153]:
# check for missing zip_code values percentage
(cleaned['zip_code'].isna().sum() / cleaned.shape[0]) * 100

27.43139281203353

In [154]:
# find mode of zip code from each borough
zip_code_mode = cleaned.groupby('borough')['zip_code'].agg(lambda x: x.mode().values[0])

# display
zip_code_mode

borough
bronx            10467
brooklyn         11208
manhattan        10029
other            10701
queens           11368
staten island    10314
Name: zip_code, dtype: object

In [155]:
# fill missing zip code with mode of zip code from each borough
cleaned['zip_code'] = cleaned.apply(lambda x: zip_code_mode[x['borough']] if x['zip_code'] is None else x['zip_code'], axis=1)

# display
cleaned.head()

,breed,spay_neuter,borough,zip_code,year,month,day,day_of_week
0,unknown,False,brooklyn,11220,2018,1,1,0
1,unknown,False,brooklyn,11208,2018,1,4,3
2,pit bull,False,brooklyn,11224,2018,1,6,5
3,mixed/other,False,brooklyn,11231,2018,1,8,0
4,pit bull,False,brooklyn,11224,2018,1,9,1


In [156]:
# check for missing zip_code values percentage
(cleaned['zip_code'].isna().sum() / cleaned.shape[0]) * 100

0.0

In [157]:
# display non-numeric zip_code values
cleaned[~cleaned['zip_code'].str.isnumeric()]

,breed,spay_neuter,borough,zip_code,year,month,day,day_of_week
21303,unknown,False,queens,?,2017,7,11,1
25122,pit bull,False,bronx,1o458,2022,9,3,5


In [158]:
# manually clean non-numeric zip_code values
cleaned.loc[cleaned['zip_code'] == '?', 'zip_code'] = zip_code_mode['queens']
cleaned.loc[cleaned['zip_code'] == '1o458', 'zip_code'] = zip_code_mode['bronx']

In [159]:
# display non-numeric zip_code values
cleaned[~cleaned['zip_code'].str.isnumeric()]

,breed,spay_neuter,borough,zip_code,year,month,day,day_of_week


In [160]:
# convert zip_code to numeric
cleaned['zip_code'] = cleaned['zip_code'].astype('int')

# display
cleaned.head()

,breed,spay_neuter,borough,zip_code,year,month,day,day_of_week
0,unknown,False,brooklyn,11220,2018,1,1,0
1,unknown,False,brooklyn,11208,2018,1,4,3
2,pit bull,False,brooklyn,11224,2018,1,6,5
3,mixed/other,False,brooklyn,11231,2018,1,8,0
4,pit bull,False,brooklyn,11224,2018,1,9,1


# Wrangling Borough
First because some data might be invalid.  
Remove 'other' because it refers to report outside of NYC.  
One-hot encoding.  

In [161]:
# borough ratios
cleaned['borough'].value_counts(normalize=True)

borough
queens           0.256172
manhattan        0.232748
brooklyn         0.218089
bronx            0.167451
staten island    0.081908
other            0.043633
Name: proportion, dtype: float64

In [162]:
# remove rows with 'other'
cleaned = cleaned[cleaned['borough'] != 'other']

# borough counts
cleaned['borough'].value_counts(normalize=True)

borough
queens           0.267859
manhattan        0.243367
brooklyn         0.228039
bronx            0.175091
staten island    0.085645
Name: proportion, dtype: float64

In [163]:
# one hot encode borough without prefix or underscore
cleaned = pd.get_dummies(cleaned, columns=['borough'], prefix='', prefix_sep='')

# display
cleaned.head()

,breed,spay_neuter,zip_code,year,month,day,day_of_week,bronx,brooklyn,manhattan,queens,staten island
0,unknown,False,11220,2018,1,1,0,False,True,False,False,False
1,unknown,False,11208,2018,1,4,3,False,True,False,False,False
2,pit bull,False,11224,2018,1,6,5,False,True,False,False,False
3,mixed/other,False,11231,2018,1,8,0,False,True,False,False,False
4,pit bull,False,11224,2018,1,9,1,False,True,False,False,False


# Wrangling breed
Take top 10 breeds for multi-hot encoding.

In [164]:
# dataframe for wrangling breed
breed_values = cleaned['breed'].copy()

# separate breed by '/', ',' and ' '
breed_values = breed_values.str.split('/')
breed_values = breed_values.apply(lambda x: [y.strip() for y in x] if x is not None else x)

# remove useless words
for word in useless_breed_words:
    breed_values = breed_values.apply(lambda x: [y.replace(word, '').strip() for y in x] if x is not None else x)

# remove white spaces
breed_values = breed_values.apply(lambda x: [y.strip() for y in x] if x is not None else x)

# map breed names to standard names
breed_values = breed_values.apply(lambda x: [breed_mapping.get(y, y) for y in x] if x is not None else x)

# display
breed_values.head()

0           [other]
1           [other]
2        [pit bull]
3    [other, other]
4        [pit bull]
Name: breed, dtype: object

In [165]:
# display breed counts
breed_counts = breed_values.explode().value_counts()
breed_counts.head()

breed
pit bull           6976
other              5245
shih tzu            983
chihuahua           936
german shepherd     812
Name: count, dtype: int64

In [ ]:
# top 10 breeds based on count, with others
top_10_breeds = breed_counts.head(11).index.tolist()
top_10_breeds

['pit bull',
 'other',
 'shih tzu',
 'chihuahua',
 'german shepherd',
 'labrador retriever',
 'bull dog',
 'yorkshire terrier',
 'maltese',
 'standard poodle',
 'husky']

In [167]:
# convert breed to top 10 breeds, with others
breed_values = breed_values.apply(lambda x: [y if y in top_10_breeds else 'other' for y in x] if x is not None else x)

# display
breed_values.head()

0           [other]
1           [other]
2        [pit bull]
3    [other, other]
4        [pit bull]
Name: breed, dtype: object

In [168]:
# multi-hot encode breed
for breed in top_10_breeds:
    cleaned[breed] = breed_values.apply(lambda x: breed in x if x is not None else False)

# drop breed column
cleaned = cleaned.drop(columns='breed')

# display
cleaned.head()

,spay_neuter,zip_code,year,month,day,day_of_week,bronx,brooklyn,manhattan,queens,...,other,shih tzu,chihuahua,german shepherd,labrador retriever,bull dog,yorkshire terrier,maltese,standard poodle,husky
0,False,11220,2018,1,1,0,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,11208,2018,1,4,3,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
2,False,11224,2018,1,6,5,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,11231,2018,1,8,0,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,11224,2018,1,9,1,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


# Exort Data

In [169]:
# convert column names to snake_case
cleaned.columns = cleaned.columns.str.replace(' ', '_')

In [170]:
# save cleaned data
cleaned.to_csv('../data/processed/dog_bite_cleaned.csv', index=False)